In [ ]:
!
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "./Final_Model/working/final_model"

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)

model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
from transformers import LogitsProcessorList, NoBadWordsLogitsProcessor

bad_words = [[t] for t in tokenizer.encode("A:", add_special_tokens=False)]
processor = LogitsProcessorList([NoBadWordsLogitsProcessor(bad_words_ids=bad_words)])



def generate(user_input):
    prompt = f"Q: {user_input}\nA:"
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_length = input_ids["input_ids"].shape[1]  # Length of the prompt

    with torch.no_grad():
        outputs = model.generate(
            **input_ids,
            max_new_tokens=500,
            
            # min_new_tokens=25,
            do_sample=True,
            temperature=0.2,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            no_repeat_ngram_size=10,    # reduce repetition
            logits_processor=processor,
            top_p=0.9
        )

    # Slice off the prompt part
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    return response


while(True):
    user_input=input("How can i help you: ")
    if(user_input=="bye" or user_input=="quit"):
        break

    response=generate(user_input)
    print(response)
    print()





